In [1]:
from fastai.text.all import *

In [2]:
torch.cuda.is_available()

True

#### Fine-tuning a language model on Rotten Tomatoes Movie Reviews
Data source: https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews/overview

In [28]:
from IPython.display import Image
Image(url='https://docs.fast.ai/images/ulmfit.png')

In [3]:
path = Path("..") / "data/sentiment-analysis-on-movie-reviews"

In [4]:
BATCH_SIZE = 32
dls_lm = TextDataLoaders.from_csv(path, csv_fname="train.tsv", delimiter="\t", text_col=2, label_col=3, is_lm=True, bs=BATCH_SIZE, valid_pct=0.1)

/home/lazylearner/anaconda3/envs/amazonreviews/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
dls_lm.show_batch(max_n=5)

,text,text_
0,xxbos by two actresses in their 50s working xxbos does n't this film have that an impressionable kid could n't stand to hear ? xxbos the toilet xxbos how well it holds up in an era in which computer - generated images are the norm xxbos a surprisingly sensitive script co - written xxbos been thoroughly debated in the media xxbos as fresh xxbos usual tropes xxbos freshly xxbos a modicum of,by two actresses in their 50s working xxbos does n't this film have that an impressionable kid could n't stand to hear ? xxbos the toilet xxbos how well it holds up in an era in which computer - generated images are the norm xxbos a surprisingly sensitive script co - written xxbos been thoroughly debated in the media xxbos as fresh xxbos usual tropes xxbos freshly xxbos a modicum of patience
1,"nice coffee table book xxbos mild escapism xxbos pacing xxbos xxmaj such a premise xxbos should now xxbos uncool the only thing missing is the ` ` xxmaj gadzooks ! '' xxbos xxmaj the sequel is everything the original was not : contrived , overblown and tie - in ready . xxbos xxmaj despite an overwrought ending xxbos the cost xxbos xxmaj shrewd but pointless . xxbos xxmaj not all of the","coffee table book xxbos mild escapism xxbos pacing xxbos xxmaj such a premise xxbos should now xxbos uncool the only thing missing is the ` ` xxmaj gadzooks ! '' xxbos xxmaj the sequel is everything the original was not : contrived , overblown and tie - in ready . xxbos xxmaj despite an overwrought ending xxbos the cost xxbos xxmaj shrewd but pointless . xxbos xxmaj not all of the stories"
2,simple in form but rich with human events . xxbos paid a matinee price xxbos xxmaj large budget xxbos gives xxmaj hollywood sequels a bad name xxbos believe it xxbos an unbalanced mixture xxbos xxmaj one of those exceedingly rare films in which the talk alone is enough to keep us xxbos xxmaj most consumers xxbos xxmaj fluffy and disposible xxbos of the worst possibilities of mankind xxbos filled with crude humor,in form but rich with human events . xxbos paid a matinee price xxbos xxmaj large budget xxbos gives xxmaj hollywood sequels a bad name xxbos believe it xxbos an unbalanced mixture xxbos xxmaj one of those exceedingly rare films in which the talk alone is enough to keep us xxbos xxmaj most consumers xxbos xxmaj fluffy and disposible xxbos of the worst possibilities of mankind xxbos filled with crude humor and
3,"gross romanticization of the delusional personality type xxbos this film , xxbos ` xxmaj silence of the xxmaj lambs ' better than ` xxmaj hannibal ' xxbos takes a potentially trite and overused concept -lrb- aliens come to xxmaj earth -rrb- and infuses it into a rustic , realistic , and altogether creepy tale of hidden invasion xxbos be distasteful to children and adults alike xxbos are intriguing and realistic xxbos technical","romanticization of the delusional personality type xxbos this film , xxbos ` xxmaj silence of the xxmaj lambs ' better than ` xxmaj hannibal ' xxbos takes a potentially trite and overused concept -lrb- aliens come to xxmaj earth -rrb- and infuses it into a rustic , realistic , and altogether creepy tale of hidden invasion xxbos be distasteful to children and adults alike xxbos are intriguing and realistic xxbos technical skill"
4,"keep us riveted to our seats . xxbos her quiet blue eyes xxbos still works . xxbos funny moment xxbos filmgoers xxbos manages , with terrific flair , to keep the extremes of screwball farce and blood - curdling family intensity on one continuum xxbos of xxmaj pokemon xxbos , its writer - director 's heart is in the right place , his plea for democracy and civic action laudable . xxbos","us riveted to our seats . xxbos her quiet blue eyes xxbos still works . xxbos funny moment xxbos filmgoers xxbos manages , with terrific flair , to keep the extremes of screwball farce and blood - curdling family intensity on one continuum xxbos of xxmaj pokemon xxbos , its writer - director 's heart is in the right pl

In [6]:
dls_lm.bs

32

In [7]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [8]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(15280, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(15280, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=15280, bias=True)
    (output_dp): RNNDropout()
  )
)

In [11]:
TEXT = "I hated this movie"
N_WORDS = 30
N_SENTENCES = 2

In [12]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

i hated this movie like The King of Lived and Bill , Prince of Egypt , and the Shakespeare Trilogy . The film would be
i hated this movie long enough to have him released it without a rage or even losing the fame . In his Death and State Video of the Century


#### Train only the head of the model while the body stays frozen.

In [14]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.319363,3.955143,0.332628,52.203182,02:37


In [15]:
learn.save('1epoch-rt')

Path('../data/sentiment-analysis-on-movie-reviews/models/1epoch-rt.pth')

In [16]:
# learn = learn.load('1epoch')

#### Fine-tune the model after unfreezing

In [17]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.880170,3.570047,0.376660,35.518257,03:06
1,3.488851,3.077907,0.451082,21.712912,03:02
2,3.073153,2.663300,0.523411,14.343538,03:02
3,2.747096,2.386356,0.570125,10.873794,03:05
4,2.511853,2.222834,0.599615,9.233460,03:02
5,2.376691,2.115220,0.618539,8.291410,03:01
6,2.253767,2.045086,0.630296,7.729821,03:03
7,2.164230,2.006582,0.638123,7.437850,03:03
8,2.124313,1.992255,0.641198,7.332046,03:05
9,2.120817,1.989486,0.641504,7.311777,03:07


In [18]:
learn.save_encoder('finetuned-rt')

In [19]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [20]:
print("\n".join(preds))

i liked this movie because i had two guys out of me in the series takes on a whole other meaning and His Secret Life enters the land of unintentional melodrama and tiresome love triangles the word ` `
i liked this movie because i ' ve seen ` jackass : the movie in the bitter end The Husband , The Wife and The Kidnapper experience , especially The filmmakers have crafted a


In [21]:
TEXT = "I hated this movie"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [22]:
print("\n".join(preds))

i hated this movie will enjoy it Though Storytelling is far more appealing Mr . Polanski 's film is a fascinating piece of history . the Russian word be what 's attracting audiences to
i hated this movie and i felt myself powerfully drawn toward the light -- the light of the exit sign . with a touch of John Woo bullet ballet To the end of the movie , they 're all
